In [ ]:
pip install langchain langchain_openai langchain_community chromadb

In [ ]:
import os
#OpenAI에서 발급받은 API Key를 'sk-...' 부분에 기입합니다.
os.environ["OPENAI_API_KEY"] = 'sk-...'

In [ ]:

# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)


In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 문서 로더 설정
loaders = [TextLoader("/content/How_to_invest_money.txt")]

docs = []
for loader in loaders:
    docs.extend(loader.load())

In [ ]:
# 문서 생성을 위한 텍스트 분할기 정의
recursive_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# 문서 분할
split_docs = recursive_splitter.split_documents(docs)

# OpenAIEmbeddings 인스턴스 생성
embeddings = OpenAIEmbeddings()

# Chroma vectorstore 생성
vectorstore = Chroma.from_documents(documents=split_docs, embedding=embeddings)

In [ ]:
from langchain.retrievers import MultiQueryRetriever
from langchain_openai import ChatOpenAI

# LLM 모델 설정 (여기서는 ChatOpenAI 사용)
llm = ChatOpenAI(model = "gpt-4o",temperature=0.2)

# MultiQueryRetriever 실행
retriever = MultiQueryRetriever.from_llm(
retriever=vectorstore.as_retriever(), # 기본 검색기 (벡터 데이터베이스)
llm=llm, # 앞서 정의한 llm (gpt-4o)
)

In [ ]:
# 샘플 질문
question = "주식 투자를 처음 시작하려면 어떻게 해야 하나요?"

# 결과 검색
unique_docs = retriever.invoke(question)
print(f"\n결과: {len(unique_docs)}개의 문서가 검색되었습니다.")